# Vector Database with Postgres

pgvector is an extension for PostgreSQL that enables the storage and manipulation of vector data directly within the database. This allows you to perform advanced vector operations, such as similarity searches, nearest neighbor searches, and clustering, all within the familiar SQL environment of PostgreSQL. It's particularly useful for applications involving machine learning, natural language processing, and other tasks where vector data is prevalent.

In [1]:
# Install Required Modules
! pip install openai psycopg2 wget

In [2]:
# Wait for Postgres to be ready
! while ! docker exec -it postgres-pgvector pg_isready -U postgres; do sleep 1; done

/var/run/postgresql:5432 - accepting connections

What's next?
  Try Docker Debug for seamless, persistent debugging tools in any container or image → docker debug postgres-pgvector
  Learn more at https://docs.docker.com/go/debug-cli/


In [3]:
# Run the command below to start PostgreSQL with pgvector in Docker:
#! docker run --name pgvector -e POSTGRES_PASSWORD=postgres -p 5432:5432 -d pgvector/pgvector:0.1.0

In [4]:
# Enable the pgvector extension by connecting to the database instance from within the container with psql and running the CREATE EXTENSION vector command:
! docker exec -it postgres-pgvector psql -U postgres -d postgres -c "CREATE EXTENSION vector"

ERROR:  extension "vector" already exists

What's next?
  Try Docker Debug for seamless, persistent debugging tools in any container or image → docker debug postgres-pgvector
  Learn more at https://docs.docker.com/go/debug-cli/


In [5]:
#%pip install openai

import os
import openai
from getpass import getpass

In [6]:
import os
openai_key = os.getenv('YOUR_API_KEY')

if (openai_key == None):
    openai_key = getpass('Provide your OpenAI API key: ')

if (not openai_key):
    raise Exception('No OpenAI API key provided. Please set the OPENAI_API_KEY environment variable or provide it when prompted.')

openai.api_key = openai_key

print('OpenAI API key set.')

OpenAI API key set.


The Movies Dataset With Embeddings

This is a movies dataset with over 45,000 movies and 26 million ratings from over 270,000 users. The original data was taken from Kaggle and updated in the following way:

In [7]:
#pip install wget

In [8]:
import requests

schema_file = "https://huggingface.co/datasets/denismagda/movies/raw/main/movie_schema.sql"
data_file = "https://huggingface.co/datasets/denismagda/movies/resolve/main/movie_data_with_openai_embeddings.sql"

print('Downloading the schema file...')
schema_response = requests.get(schema_file, verify=False)
with open('movie_schema.sql', 'wb') as file:
    file.write(schema_response.content)

print('Downloading the data file...')
data_response = requests.get(data_file, verify=False)
with open('movie_data_with_openai_embeddings.sql', 'wb') as file:
    file.write(data_response.content)

print('Finished downloading the files.')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdn-lfs-us-1.huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Finished downloading the files.


In [9]:
#%pip install psycopg2-binary

In [10]:
#pip install --force-reinstall psycopg2

In [19]:
import psycopg2
from psycopg2 import sql

print('Connecting to PostgreSQL...')
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=password")

# Create a cursor object
cursor = conn.cursor()

print('Successfully connected to PostgreSQL.')

def execute_sql_script(cursor, file_path):
    with open(file_path, 'r') as file:
        sql_commands = file.read().split(';')  # Split commands by semicolon
        for command in sql_commands:
            command = command.strip()
            if command:  # Skip empty commands
                cursor.execute(sql.SQL(command))

# Create the schema
print('Creating the schema...')
execute_sql_script(cursor, 'movies_schema.sql')
conn.commit()

# Load the data
print('Loading the data. It might take a minute...')
execute_sql_script(cursor, 'movie_data_with_openai_embedded.sql')
conn.commit()

# Verify the data load
cursor.execute('SELECT COUNT(*) FROM movie')
result = cursor.fetchone()

print(f'{result[0]} movies loaded.')

# Close the cursor and connection
cursor.close()
conn.close()

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.
Creating the schema...
Loading the data. It might take a minute...
13 movies loaded.


#### Perform Vector Similarity Search

In [22]:
# Converting the prompt to the pgvector embedding
def get_embedding(prompt):
    response = openai.embeddings.create(
        input=prompt,
        model='text-embedding-ada-002')

    embedding = response.data[0].embedding

    # Converting the embedding to the pgvector and returning it
    return '[' + ','.join(map(str, embedding)) + ']'

# Getting the matching threshold for the similarity search
def get_matching_threshold():
    return 0.7

# Getting the number of matching movies to return
def get_matching_count():
    return 3

# Second, find the most relevant movies for a provided user prompt by calculating 
# the cosine distance (<=>) between the prompt's and movies' embeddings:

user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'SELECT title, overview '
    'FROM movie WHERE 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [24]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'EXPLAIN (costs off) SELECT title, overview '
    'FROM movie WHERE 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}